---

* 출처: LangChain 공식 문서 또는 해당 교재명
* 원본 URL: https://smith.langchain.com/hub/teddynote/summary-stuff-documents

---

## **OutputFixingParser**

* `OutputFixingParser`는 출력 파싱 과정에서 발생할 수 있는 `오류`를 `자동`으로 `수정`하는 기능 제공

* 설계 방식: `다른 파서` (예컨대 PydanticOutputParser)를 `래핑`하고, 이 파서가 `처리할 수 없는 형식`의 출력이나 `오류`를 반환할 경우, `추가적인 LLM 호출`을 통해 `오류`를 `수정`하도록 설계됨

### **핵심**

* 첫 번째 시도에서 스키마를 준수하지 않는 결과가 나올 경우 → `OutputFixingParser`가 `자동`으로 형식이 `잘못된 출력`을 `인식` → 이를 `수정`하기 위한 `새로운 명령어`와 함께 `모델에 다시 제출`
  
  * **수정을 위한 명령어는 오류를 정확히 지적** 

  * **올바른 형식으로 데이터를 재구성할 수 있도록 구체적인 지시를 포함해야 함**

### **사용 방법**

* 예시: `PydanticOutputParser` → 특정 데이터 스키마를 준수하는 출력을 생성하고자 함 → `일부 필드`가 `누락` or `데이터 유형`이 `잘못된 경우`가 발생할 수 있음

* 이때 `OutputFixingParser` = `다음 단계` → `해당 오류`를 `수정`하는 `지시`를 포함한 `새로운 요청`을 `LLM`에 `제출` → `LLM` = `오류`를 `수정`한 **`새로운 출력`을 생성**

In [2]:
# 기본 임포트
from langsmith import Client
from langchain.prompts import PromptTemplate                            
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from typing import List

from langsmith import Client

import os
import json

# 클라이언트 생성 
api_key = os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=api_key)

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from pydantic import BaseModel, Field
from langsmith import traceable                                     # LangSmith 추적 설정

# LLM 초기화
gemini_lc = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite",
        temperature=0.7,                                    
        max_output_tokens=4096,
    )

In [4]:
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(
        description="list of names of films they starred in")


actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

---

### **잘못된 형식을 먼저 입력해보기**

In [ ]:
# 잘못된 형식을 일부러 입력
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

# 잘못된 형식으로 입력된 데이터를 파싱하려고 시도
parser.parse(misformatted)

# 오류 출력


<small>

* 셀 출력 (0.4s)

    ```python
    ---------------------------------------------------------------------------
    JSONDecodeError                           Traceback (most recent call last)
    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain_core/output_parsers/json.py:82, in JsonOutputParser.parse_result(self, result, partial)
        81 try:
    ---> 82     return parse_json_markdown(text)
        83 except JSONDecodeError as e:

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain_core/utils/json.py:150, in parse_json_markdown(json_string, parser)
        149     json_str = json_string if match is None else match.group(2)
    --> 150 return _parse_json(json_str, parser=parser)

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain_core/utils/json.py:166, in _parse_json(json_str, parser)
        165 # Parse the JSON string into a Python dictionary
    --> 166 return parser(json_str)

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain_core/utils/json.py:123, in parse_partial_json(s, strict)
        120 # If we got here, we ran out of characters to remove
        121 # and still couldn't parse the string as JSON, so return the parse error
        122 # for the original string.
    --> 123 return json.loads(s, strict=strict)

    File ~/.pyenv/versions/3.13.5/lib/python3.13/json/__init__.py:359, in loads(s, cls, object_hook, parse_float, parse_int, parse_constant, object_pairs_hook, **kw)
        358     kw['parse_constant'] = parse_constant
    --> 359 return cls(**kw).decode(s)

    File ~/.pyenv/versions/3.13.5/lib/python3.13/json/decoder.py:345, in JSONDecoder.decode(self, s, _w)
        341 """Return the Python representation of ``s`` (a ``str`` instance
        342 containing a JSON document).
        343 
        344 """
    --> 345 obj, end = self.raw_decode(s, idx=_w(s, 0).end())
        346 end = _w(s, end).end()

    File ~/.pyenv/versions/3.13.5/lib/python3.13/json/decoder.py:361, in JSONDecoder.raw_decode(self, s, idx)
        360 try:
    --> 361     obj, end = self.scan_once(s, idx)
        362 except StopIteration as err:

    JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

    The above exception was the direct cause of the following exception:

    OutputParserException                     Traceback (most recent call last)
    Cell In[5], line 5
        2 misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"
        4 # 잘못된 형식으로 입력된 데이터를 파싱하려고 시도
    ----> 5 parser.parse(misformatted)
        7 # 오류 출력

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain_core/output_parsers/pydantic.py:77, in PydanticOutputParser.parse(self, text)
        68 def parse(self, text: str) -> TBaseModel:
        69     """Parse the output of an LLM call to a pydantic object.
        70 
        71     Args:
    (...)     75         The parsed pydantic object.
        76     """
    ---> 77     return super().parse(text)

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain_core/output_parsers/json.py:96, in JsonOutputParser.parse(self, text)
        87 def parse(self, text: str) -> Any:
        88     """Parse the output of an LLM call to a JSON object.
        89 
        90     Args:
    (...)     94         The parsed JSON object.
        95     """
    ---> 96     return self.parse_result([Generation(text=text)])

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain_core/output_parsers/pydantic.py:61, in PydanticOutputParser.parse_result(self, result, partial)
        48 """Parse the result of an LLM call to a pydantic object.
        49 
        50 Args:
    (...)     58     The parsed pydantic object.
        59 """
        60 try:
    ---> 61     json_object = super().parse_result(result)
        62     return self._parse_obj(json_object)
        63 except OutputParserException:

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain_core/output_parsers/json.py:85, in JsonOutputParser.parse_result(self, result, partial)
        83 except JSONDecodeError as e:
        84     msg = f"Invalid json output: {text}"
    ---> 85     raise OutputParserException(msg, llm_output=text) from e

    OutputParserException: Invalid json output: {'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}
    For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE
    ```

<small>

---

* 오류 메시지 해석

  * 1. **첫 번째 오류: `JSONDecodeError`**

  ```plaintext
    JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
  ```

    * **문제**: `JSON`을 파싱하려고 했는데, `JSON`의 **key**(속성 이름)이 **따옴표**로 감싸져 있지 않아서 발생한 오류
    * **구체적으로**
      * `{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}`라는 문자열에서 `name`, `film_names` 등의 key가 작은따옴표 `'`로 감싸져 있음
      * 하지만 `JSON`에서는 key가 **반드시 큰따옴표 `"`** 로 감싸져야 함
    * 이 오류는 Python의 기본 `json.loads()` 함수가 발생시키는 오류로, 작은 따옴표를 **큰따옴표** 로 바꿔야 한다는 것을 알려줌.

  ---

    * 2. **두 번째 오류: `OutputParserException`**

  ```plaintext
    The above exception was the direct cause of the following exception:

    OutputParserException: Invalid json output: {'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}
  ```

    * **문제**: `JSON` 파싱 오류 발생 → `OutputParserException` 발생
    * **구체적으로**
      * 첫 번째 오류인 `JSONDecodeError`가 발생 → 이 오류가 `OutputParserException`을 일으켰음
      * 즉, **`잘못된 JSON 형식`** 이라는 예외 발생
    * **왜?**
      * `OutputParser` = **`JSON` 형식의 데이터를 예상**
      * `but` 받은 데이터는 **`JSON` 형식이 아니었기 때문에** 이 예외가 발생

  ---

  * 3. **전체 오류 흐름**

    * **`JsonOutputParser.parse()`** 메서드가 호출되었을 때, `{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}`라는 문자열을 **잘못된 `JSON`** 형식으로 처리하려고 했고, 이 과정에서 `JSONDecodeError`가 발생
  
    * 그런 후, 이 오류가 `OutputParserException`으로 이어졌는데, 이 예외는 `langchain`의 예외 처리 시스템에 의해 더 상위에서 포착되고, 최종적으로 **잘못된 `JSON`**을 전달했다는 메시지와 함께 예외가 발생

  ---

  * 4. **문제가 되는 부분**

    * **잘못된 JSON 형식**: `{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}`라는 문자열에서 작은따옴표 `'`를 사용했기 때문에 **유효한 JSON이 아니었음**. 올바른 JSON은 \*\*큰따옴표 `"`\*\*를 사용해야 함
  
    * 예를 들어, 다음과 같이 바꿔야 정상:

      ```json
      {"name": "Tom Hanks", "film_names": ["Forrest Gump"]}
      ```

---

* 요약: **`JSONDecodeError`가 먼저 발생한 후, 그 결과로 **유효하지 않은 JSON 형식**이 전달되어 `OutputParserException`이 발생한 흐름**

  * **`JSONDecodeError`**: 작은따옴표로 감싸진 JSON을 파싱하려 해서 발생한 오류.
  * **`OutputParserException`**: 파싱 실패로 인한 예외로, 유효한 JSON을 받지 못했다고 알림.
  * **문제의 근원**: JSON 문자열에서 **작은따옴표** 대신 **큰따옴표**를 써야 한다는 것.

### **`OutputFixingParser` 사용해 바로 잡기**

In [6]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=gemini_lc)

In [ ]:
# 잘못된 형식의 출력
misformatted                    # "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

In [8]:
# OutputFixingParser 를 사용하여 잘못된 형식의 출력을 파싱
actor = new_parser.parse(misformatted)

In [ ]:
# 파싱된 결과
actor                            # Actor(name='Tom Hanks', film_names=['Forrest Gump'])

Actor(name='Tom Hanks', film_names=['Forrest Gump'])

---

* *next: LLM model*